In [ ]:
# ==================== 第一部分：导入库 ====================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("✅ 库导入成功！")

# ==================== 第二部分：生成模拟数据 ====================
# 我们不从网络获取，避免复杂问题，先用模拟数据
np.random.seed(42)  # 确保每次运行结果一样
dates = pd.date_range('2024-01-01', periods=30, freq='D')
# 模拟股价：随机游走
prices = 100 + np.cumsum(np.random.randn(30) * 2)

# 创建DataFrame
df = pd.DataFrame({'收盘价': prices}, index=dates)
print("\n📊 模拟股价数据（前5行）：")
print(df.head())

# ==================== 第三部分：基础计算 ====================
# 计算每日涨跌幅
df['涨跌幅(%)'] = df['收盘价'].pct_change() * 100

# 计算移动平均线（5日均线）
df['5日均线'] = df['收盘价'].rolling(window=5).mean()

print("\n📈 基础统计：")
print(f"• 期初价: {df['收盘价'].iloc[0]:.2f} 元")
print(f"• 期末价: {df['收盘价'].iloc[-1]:.2f} 元")
print(f"• 总收益率: {(df['收盘价'].iloc[-1]/df['收盘价'].iloc[0]-1)*100:.2f}%")
print(f"• 最大单日涨幅: {df['涨跌幅(%)'].max():.2f}%")
print(f"• 最大单日跌幅: {df['涨跌幅(%)'].min():.2f}%")

# ==================== 第四部分：可视化 ====================
plt.figure(figsize=(12, 8))

# 子图1：价格与均线
plt.subplot(3, 1, 1)
plt.plot(df.index, df['收盘价'], label='收盘价', color='blue', linewidth=2)
plt.plot(df.index, df['5日均线'], label='5日均线', color='orange', linestyle='--')
plt.title('💰 模拟股价走势与均线', fontsize=14)
plt.ylabel('价格 (元)')
plt.legend()
plt.grid(True, alpha=0.3)

# 子图2：涨跌幅
plt.subplot(3, 1, 2)
colors = ['green' if x>0 else 'red' for x in df['涨跌幅(%)']]
plt.bar(df.index, df['涨跌幅(%)'].fillna(0), color=colors, alpha=0.7)
plt.title('📈 每日涨跌幅', fontsize=14)
plt.ylabel('涨跌幅 (%)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linewidth=0.5)

# 子图3：收益率分布
plt.subplot(3, 1, 3)
plt.hist(df['涨跌幅(%)'].dropna(), bins=15, edgecolor='black', alpha=0.7)
plt.title('📊 涨跌幅分布直方图', fontsize=14)
plt.xlabel('涨跌幅 (%)')
plt.ylabel('频次')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==================== 第五部分：简单策略模拟 ====================
print("\n" + "="*50)
print("🧪 简单策略回测：5日均线上穿策略")
print("="*50)

# 生成信号：当收盘价上穿5日均线时买入（1），下穿时卖出（-1）
df['信号'] = 0
df.loc[df['收盘价'] > df['5日均线'], '信号'] = 1
df.loc[df['收盘价'] < df['5日均线'], '信号'] = -1

# 计算策略收益（简化版）
df['持仓'] = df['信号'].shift(1)  # 第二天才执行信号
df['策略每日收益'] = df['持仓'] * df['涨跌幅(%)'] / 100
df['策略累计收益'] = (1 + df['策略每日收益']).cumprod() - 1
df['基准累计收益'] = (1 + df['涨跌幅(%)']/100).cumprod() - 1

final_strategy_return = df['策略累计收益'].iloc[-1] * 100
final_benchmark_return = df['基准累计收益'].iloc[-1] * 100

print(f"📊 回测结果（30天）：")
print(f"• 策略最终收益: {final_strategy_return:.2f}%")
print(f"• 买入持有收益: {final_benchmark_return:.2f}%")
print(f"• 策略超额收益: {final_strategy_return - final_benchmark_return:.2f}%")

# 绘制收益对比
plt.figure(figsize=(10, 4))
plt.plot(df.index, df['策略累计收益']*100, label='策略收益', linewidth=2)
plt.plot(df.index, df['基准累计收益']*100, label='基准收益', linestyle='--')
plt.title('📈 策略收益 vs 基准收益', fontsize=14)
plt.ylabel('累计收益 (%)')
plt.xlabel('日期')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n🎉 恭喜！你完成了第一个完整的量化分析流程！")
print("• 数据生成 → 指标计算 → 可视化 → 策略回测 → 绩效评估")